## Environment Variables

In [ ]:
import os
import getpass
from dotenv import load_dotenv

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_DEPLOYMENT_NAME = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")
AZURE_OPENAI_VERSION = os.getenv("AZURE_OPENAI_VERSION")

os.environ["TAVILY_API_KEY"] = getpass.getpass()

print(AZURE_OPENAI_DEPLOYMENT_NAME)

## Custom Tools

In [ ]:
import random
from langchain_core.tools import tool, Tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_experimental.utilities import PythonREPL

@tool("random_number")
def random_number_maker(input: str):
    """Returns a random number between 0-100."""
    return random.randint(0, 100)

@tool("calculate")
def calculate(num1: int, num2: int, operator: str):
    """Returns the result of the calculation."""
    if operator == '+':
        result = num1 + num2
    elif operator == '-':
        result = num1 - num2
    elif operator == '*':
        result = num1 * num2
    elif operator == '/':
        result = num1 / num2
    elif operator == '^':
        result = num1 ** num2
    elif operator == '%':
        result = num1 % num2
    return result

tavily_search = TavilySearchResults(max_results=3)

python_repl = PythonREPL()
repl_tool = Tool(
    name="python_repl",
    description="""A Python shell. Use this to execute python commands. Input should be a valid python command.
                If you want to see the output of a value, you should print it out with `print(...)`.""",
    func=python_repl.run
)

tavily_search.invoke("What is Python?")

## Initialize Agent

In [ ]:
from langchain import hub
from langchain_openai import AzureChatOpenAI
from langchain.agents import create_openai_functions_agent, create_json_chat_agent, create_react_agent, AgentExecutor
# from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langgraph.prebuilt.tool_executor import ToolExecutor

llm = AzureChatOpenAI(
    openai_api_version=AZURE_OPENAI_VERSION,
    openai_api_key=AZURE_OPENAI_API_KEY,
    deployment_name=AZURE_OPENAI_DEPLOYMENT_NAME,
    azure_endpoint=AZURE_ENDPOINT,
    temperature=0,
    streaming=True
    # callbacks=[StreamingStdOutCallbackHandler()]
)

## agent_scratchpad: a function that formats the intermediate steps of the agent's actions and observations into a string
# prompt = hub.pull("hwchase17/react-chat-json")
prompt = hub.pull("hwchase17/openai-functions-agent")

tools = [random_number_maker, calculate, tavily_search, repl_tool]
tool_executor = ToolExecutor(tools)

agent_runnable = create_openai_functions_agent(llm, tools, prompt)
# agent_runnable = create_json_chat_agent(llm, tools, prompt)
# agent_runnable = create_react_agent(llm, tools, prompt)
# agent_executor = AgentExecutor(agent=agent_runnable, tools=tools, verbose=True, return_intermediate_steps=True)

prompt.messages

In [4]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

contextualize_q_system_prompt = """Given a chat history and the latest user question \
which might reference context in the chat history, formulate a standalone question \
which can be understood without the chat history. Do NOT answer the question, \
just reformulate it if needed and otherwise return it as is."""
contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
# history_aware_retriever = create_history_aware_retriever(
#     llm, retriever, contextualize_q_prompt
# )

In [ ]:
inputs = {
    "input": "List the official 7 wonders of the world.",
    "chat_history": [],
    "intermediate_steps": []
}
response = agent_runnable.invoke(inputs)
response

In [ ]:
model = llm.bind_tools(tools)

model.invoke("What is Machine Learning?")

## Making the GraphState

In [ ]:
from typing import TypedDict, Annotated, Sequence, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator

class AgentState(TypedDict):
    input: str
    chat_history: list[BaseMessage]
    agent_out: Union[AgentAction, AgentFinish, None]
    intermediate_steps: Annotated[list[tuple[AgentAction, str]], operator.add]

print("GraphState complete..")

## Define Nodes for Graph

In [ ]:
import json

def run_agent(state: list):
    print("> run_agent")
    agent_out = agent_runnable.invoke(state)
    return {"agent_out": agent_out}

def execute_tools(state: list):
    print("> execute_tools")
    agent_action = state["agent_out"]
    output = tool_executor.invoke(agent_action)
    print(f"The agent action is: {agent_action}")
    print(f"The tool result is: {output}")
    return {"intermediate_steps": [(agent_action, str(output))]}

def router(state: list):
    print("> router")
    if isinstance(state['agent_out'], AgentFinish):
        return "end"
    else:
        return "continue"

def rag_final_answer(state: list):
    print("> rag_final_answer")
    query = state["input"]
    context = state["intermediate_steps"]
    print(f"Input: {query}")
    print(f"Context: {context}")

    prompt = f"""You are a great researcher who conducts very technical research around a given
    query and context. Your research outcome only contains facts and urls to support those facts
    in that order.
    CONTEXT: {context}
    QUESTION: {query}"""
    out = model.invoke(prompt)
    return {"agent_out": out}

print("Node definitions complete..")

## Define Graph

In [ ]:
from langgraph.graph import StateGraph, END

graph = StateGraph(AgentState)

graph.add_node("agent", run_agent)
graph.add_node("tools", execute_tools)
graph.add_node("final_answer", rag_final_answer)

# conditional edges are controlled by router
graph.add_conditional_edges(
    "agent",
    router,
    {
        "continue": "tools",
        "end": "final_answer"
    }
)
graph.add_edge("tools", "agent")
graph.add_edge("final_answer", END)

graph.set_entry_point("agent")

graph_runnable = graph.compile()

print("Graph complete..")

In [ ]:
graph.branches

In [ ]:
graph.nodes, graph.edges

In [ ]:
graph.channels

In [ ]:
graph_inputs = {
    "input": "What are the results for the 4 fundamental calculations of 2 random numbers?",
    "chat_history": [],
    "intermediate_steps": []
}
graph_runnable.invoke(graph_inputs)

In [ ]:
out = graph_runnable.invoke({
    "input": "Please provide a summary regarding Canadian immigration policies.",
    "chat_history": []
})
print(out["agent_out"])

In [ ]:
out = graph_runnable.invoke({
    "input": "What was the ice hockey finals series and game score?",
    "chat_history": []
})
print(out)

In [ ]:
out = graph_runnable.invoke({
    "input": "What is the date tomorrow and the weather in Toronto then?",
    "chat_history": []
})
print(out["agent_out"])
# print(out.strip('content='))

In [ ]:
graph_runnable.invoke({
    "input": "Create a bar graph resembling a gaussian distribution and find the probability density function formula.",
    "chat_history": []
})

In [ ]:
out = graph_runnable.invoke({
    "input": "hi",
    "chat_history": []
})
print(out)

In [ ]:
out = graph_runnable.invoke({
    "input": "hi, please don't respond to me with a `source`",
    "chat_history": []
})
print(out["agent_out"])